In [2]:
import pandas as pd 
import numpy as np

In [157]:
train_g = pd.read_csv('train_groups.csv')
test_g  = pd.read_csv('test_groups.csv')
t = pd.concat([test_g, train_g], axis=0, sort=False)# объединение для получения всех айди
print(t.shape)
t.head()

(28317, 4)


,pair_id,group_id,doc_id,target
0,11691,130,6710,NaN
1,11692,130,4030,NaN
2,11693,130,5561,NaN
3,11694,130,4055,NaN
4,11695,130,4247,NaN


In [158]:
ng = pd.read_csv('ng.csv')
ng.head()

,doc_id,title
0,15731,ваз 21213 замена подшипник ступица нива
1,14829,ваз 2107 оптом сочи сравнить цены купить потре...
2,15764,купить ступица лада калина2 трансмиссия перех...
3,17669,классика 21010 21074
4,14852,ступица нива — замена подшипник свой рука


In [159]:
test = pd.merge(t, ng, on='doc_id', how='left')# сопоставление тайтд -айди
test = test.replace(np.nan, 'noinformation')
print(test.shape)
test.head()

(28317, 5)


,pair_id,group_id,doc_id,target,title
0,11691,130,6710,noinformation,прописать админк кс 16 друг youtube
1,11692,130,4030,noinformation,скачать sglrp доработка слив мода mysql rp ...
2,11693,130,5561,noinformation,прописать админк кс 16 counterstrike каталог...
3,11694,130,4055,noinformation,прописать простой админк кс 1 6
4,11695,130,4247,noinformation,подбор админовый сервер код4 архив форум ozone


In [160]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.cluster import DBSCAN

In [161]:
h_vectorizer = HashingVectorizer(n_features=2**11, dtype = 'float32' ,ngram_range=(1, 3))
X = h_vectorizer.fit_transform(test.title) # постороение хэшей на файле без пропусков
h_vec = pd.DataFrame(X.toarray())
h_vec.shape

(28317, 2048)

In [162]:
a = pd.concat([test, h_vec], axis = 1)

In [163]:
b = a[a.group_id > 129].sort_values(by='group_id')


# DBSCAN

In [164]:
db_res = [] 

In [165]:
dbs = DBSCAN(eps=0.7, min_samples=5, metric='cosine',  algorithm='auto') 

In [166]:
for i in range(130, 310):
    g = b[b.group_id == i]
    label = dbs.fit_predict(g.drop(['pair_id', 'group_id', 'doc_id', 'title', 'target'], 1))
    db_res= np.concatenate((db_res, label), axis = None)
    

In [167]:
db_res

array([ 0., -1., -1., ...,  0.,  0.,  0.])

In [168]:
np.unique(db_res, return_counts=True)

(array([-1.,  0.,  1.,  2.,  3.]), array([11941,  4268,   369,    44,     5]))

In [171]:
db_data = b.copy()
db_data['db_res'] = db_res
db_data = db_data[['pair_id', 'group_id', 'doc_id', 'title', 'db_res']]

In [191]:
db_data['target'] = db_data.db_res
db_data[['pair_id', 'target']].to_csv('all_hash_dbscan.csv', index=False)

# AgglomerativeClustering

In [173]:
from sklearn.cluster import AgglomerativeClustering

In [174]:
agg = AgglomerativeClustering(n_clusters=2, affinity='euclidean')
agg_res = []

In [175]:
for i in range(130, 310):
    g = b[b.group_id == i]
    label = agg.fit_predict(g.drop(['pair_id', 'group_id', 'doc_id', 'title', 'target'], 1))
    agg_res= np.concatenate((agg_res, label), axis = None) 

In [176]:
agg_data = b.copy()
agg_data['agg_res'] = agg_res
agg_data = agg_data[['pair_id', 'group_id', 'doc_id', 'title', 'agg_res']]

In [190]:
agg_data['target'] = agg_data.agg_res
agg_data[['pair_id', 'target']].to_csv('all_hash_agg.csv', index=False)

# свой хэш для каждой группы

In [178]:
h_vectorizer = HashingVectorizer(n_features=2**11, dtype = 'float32' ,ngram_range=(1, 3))

In [185]:
groups = pd.DataFrame()

In [186]:
for i in range(130, 310):
    g = test[test.group_id == i]
    X = h_vectorizer.fit_transform(g.title)
    vec = pd.DataFrame(X.toarray())
    label = agg.fit_predict(vec)
    g.loc[:,'label'] = label
    groups = pd.concat([groups, g], axis=0)

/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


...

/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/dmitriy/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [189]:
groups = groups.drop('target', 1)
groups['target'] =groups.label
groups[['pair_id', 'target']].to_csv('group_hash_agg.csv', index=False)

In [188]:
groups.head()

,pair_id,group_id,doc_id,title,label,target
0,11691,130,6710,прописать админк кс 16 друг youtube,0,0
1,11692,130,4030,скачать sglrp доработка слив мода mysql rp ...,1,1
2,11693,130,5561,прописать админк кс 16 counterstrike каталог...,0,0
3,11694,130,4055,прописать простой админк кс 1 6,0,0
4,11695,130,4247,подбор админовый сервер код4 архив форум ozone,1,1


In [148]:
label = 0
for i in range(130, 310):
    g = test[test.group_id == i]
    X = h_vectorizer.fit_transform(g.title)
    vec = pd.DataFrame(X.toarray())
    s = agg.fit_predict(vec)
    label= np.concatenate((label, s), axis = None)

In [192]:
aaa = pd.merge(groups[['pair_id', 'target']], agg_data, how='inner', on='pair_id')

In [194]:
# aaa

In [203]:
fin = []
for i in range(130, 310):
    g = groups[groups.group_id == i]
    m = g.label
#     print(m)
    s = g.label.sum()
    if s > len(g) / 2:
        m = [0 if x == 1 else 1 for x in m]
#     print(m)
    fin = np.concatenate((fin, m), axis = None)
        

In [213]:
len(fin)
fin = np.asarray(fin, dtype=int)

In [214]:
groups = groups.drop('target', 1)
groups['target'] = fin

In [215]:
groups

,pair_id,group_id,doc_id,title,label,target
0,11691,130,6710,прописать админк кс 16 друг youtube,0,1
1,11692,130,4030,скачать sglrp доработка слив мода mysql rp ...,1,0
2,11693,130,5561,прописать админк кс 16 counterstrike каталог...,0,1
3,11694,130,4055,прописать простой админк кс 1 6,0,1
4,11695,130,4247,подбор админовый сервер код4 архив форум ozone,1,0
5,11696,130,5983,каталог стать the best original portal in ukr...,1,0
6,11697,130,5784,eugene kirian » блог,1,0
7,11698,130,4700,файл htaccess основной параметры увеличивать б...,1,0
8,11699,130,4093,дать админк cs 16 хостинг my arena ответ игра ...,1,0
9,11700,130,6487,joomla 25 убирать хлебный крошка выбрать страница,1,0


In [216]:
groups[['pair_id', 'target']].to_csv('group_hash_agg.csv', index=False)